# Retention: Comparing kernel to current redshift pulls
* StelllarAlgo Data Science
* Ryan Kazmerik & Grant Donst
* Apr 3, 2023

In [26]:
import awswrangler as wr
import pandas as pd

from data_sci_toolkit.aws_tools import permission_tools 
from data_sci_toolkit.aws_tools import redshift_tools

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [27]:
SESSION = permission_tools.get_aws_session("Dev-DataScienceAdmin")

Authorized as AROAQBUNWK7XN5DTYWPLJ:gdonst@stellaralgo.com


In [28]:
DATABASE = "integrations_milb_aviators"
S3_BUCKET = "s3://dev-athena-testing-us-east-1"

In [29]:
df2 = wr.athena.read_sql_query(
    boto3_session= SESSION, 
    ctas_approach= False,
    database= DATABASE,
    s3_output= S3_BUCKET,
    sql= """
        WITH customers as (
            select * 
            from integrations_milb_aviators.customers c
            CROSS JOIN UNNEST(account_ids['ticketing'], account_ids['retail']) as t(ticket_ids, retail_ids)
        ),
        ticketing AS (
            select *
            from integrations_milb_aviators.tickets t
            CROSS JOIN UNNEST(resales, forwards, scans) as t(resale_data, forward_data, scan_data)
            where returned_at IS NULL
        )
        
        select * 
        from ticketing t
        INNER JOIN integrations_milb_aviators.ticketing_events e
        on t.event_id = e.id
        INNER JOIN customers c
        on c.ticket_ids = t.account_id
    """
)
df2.shape

In [ ]:
df2.head()

,source_system,source_system_group,stlr_reported_revenue,client_property,stlr_product,seat_id,section_name,row_name,seat_num,season_id,event_id,event_name,plan_event_id,ticket_type,comp_code,promo_code,account_id,price_code,price,account_rep_id,account_rep_name,purchased_at,returned_at,updated_at,forwards,resales,scans,resale_data,forward_data,scan_data,source_system.1,source_system_group.1,id,stlr_is_renewal,stlr_game_number,client_property.1,name,full_name,venue,team,start_date,end_date,day_of_week,time,time_of_day,event_type,season_id.1,season,season_year,game_number,min_events,total_events,fse,has_payment_plan,major_category,minor_category,id.1,account_ids,email,first_name,last_name,address,city,state,country,zip,phone,company_name,account_type,account_rep_name.1,demographics,scv_history,scv_updated_at,venues,ticket_ids,retail_ids
0,ticketmaster,ticketing,10.0,milb_aviators,Individual,77C686B4755C54AB30E77C708564D07B,GA3,GA0,16,109,4175,EV220812,<NA>,Adult,0,<NA>,1955298,A*,10.0,14,Michelle Taggart,2022-08-03 19:55:45.850,NaT,NaT,NaN,NaN,"[{account_id=1955298, scanned_at=2022-08-14 00...",NaN,NaN,"{account_id=1955298, scanned_at=2022-08-14 00:...",ticketmaster,ticketing,4175,True,<NA>,other,EV220812,Field Of Dreams,Las Vegas Ballpark,The Sandlot,2022-08-12 06:00:00,2022-08-12 06:00:00,FRI,19:00:00,Evening,Single Event,109,2022 Flicks On The Field - Strikeout Summer Se...,2022,3,1,1,0.0,False,Movies,Comedy,b746f41d884f8784de2aee399f8c9551,{ticketing=[1955298]},frait_b@yahoo.com,Wesley,Bailey,<NA>,<NA>,<NA>,United States,89118,<NA>,<NA>,Personal,Michelle Taggart,NaN,[],2023-03-23 18:43:28,"[{name=Las Vegas Ballpark, distance=12.7510290...",1955298,<NA>
1,ticketmaster,ticketing,20.0,milb_aviators,Individual,58364A3536A0D2B66A08AEB8E305D100,GA85,GA6,23,106,4162,EV220329,<NA>,Adult,0,<NA>,2404686,A*,20.0,14,Michelle Taggart,2022-03-20 18:25:21.200,NaT,NaT,NaN,NaN,"[{account_id=2404686, scanned_at=null, scanned...",NaN,NaN,"{account_id=2404686, scanned_at=null, scanned_...",ticketmaster,ticketing,4162,False,5,other,EV220329,Asu Vs Unlv,Las Vegas Ballpark,ASU vs UNLV,2022-03-29 06:00:00,2022-03-29 06:00:00,TUE,18:05:00,Evening,Single Event,106,2022 Unlv Baseball,2022,1,1,1,0.0,False,Sports,College Baseball,7d8fdd27b8370bc3766d8822feb9e94c,{ticketing=[2404686]},kristacolford@telus.net,Krista,Colford,3829 52 Ave,Drayton Valley,Alberta,Canada,T7A0B7,+1 (780) 898-2903,<NA>,Personal,Michelle Taggart,NaN,[],2023-03-23 18:43:28,"[{name=Las Vegas Ballpark, distance=1897.69334...",2404686,<NA>
2,ticketmaster,ticketing,20.0,milb_aviators,Individual,5ECA88C6EEFE3F0D7FBFD56881750255,GA86,GA4,14,106,4169,EV220506,<NA>,Adult,0,<NA>,2562384,A*,20.0,14,Michelle Taggart,2022-05-06 09:10:05.760,NaT,NaT,NaN,NaN,"[{account_id=2562384, scanned_at=null, scanned...",NaN,NaN,"{account_id=2562384, scanned_at=null, scanned_...",ticketmaster,ticketing,4169,False,24,other,EV220506,Hawaii Vs Unlv,Las Vegas Ballpark,HAWAII vs UNLV,2022-05-06 06:00:00,2022-05-06 06:00:00,FRI,18:05:00,Evening,Single Event,106,2022 Unlv Baseball,2022,2,1,1,0.0,False,Sports,College Baseball,1665b0f427d76d3bb74fef0cf49e5590,{ticketing=[2562384]},vegasjack55@gmail.com,Frank,Romero,5505 Lochmor Ave,Las Vegas,Nevada,United States,89130,+1 (714) 402-5076,<NA>,Personal,Michelle Taggart,NaN,[],2023-03-23 18:43:28,"[{name=Las Vegas Ballpark, distance=14.2902050...",2562384,<NA>
3,ticketmaster,ticketing,20.0,milb_aviators,Individual,E507DA8A6FD65CDEF6A16F56125F278E,GA86,GA0,31,106,4170,EV220507,<NA>,Adult,0,<NA>,2507674,A*,20.0,14,Michelle Taggart,2022-04-25 17:17:17.790,NaT,NaT,NaN,NaN,"[{account_id=2507674, scanned_at=null, scanned...",NaN,NaN,"{account_id=2507674, scanned_at=null, scanned_...",ticketmaster,ticketing,4170,False,25,other,EV220507,Hawaii Vs Unlv,Las Vegas Ballpark,HAWAII vs UNLV,2022-05-07 06:00:00,2022-05-07 06:00:00,SAT,13:05:00,Afternoon,Single Event,106,2022 Unlv Baseball,2022,3,1,1,0.0,False,Sports,College Baseball,232c0401ac6bba05e45befb9dc0b6b22,{ticketing=[2507674

In [ ]:
sql3 = """with 
customers as (
    select c.id as scv_id, source_system_type, account_id, venue
    from integrations_milb_aviators.customers as c
    cross join unnest(account_ids) as t(source_system_type, source_account_ids)
    cross join unnest(source_account_ids) as a(account_id)
    cross join unnest(venues) as v(venue)
),
ticketing_customers as (
    select 
        c.scv_id, 
        c.venue.distance as distance_to_venue, 
        te.season_year, 
        t.event_id,
        te.start_date as event_date,
        t.stlr_product, 
        t.seat_id,
        t.price,
        t.purchased_at,
        t.scans is not null as was_attended
    from customers as c
    join tickets as t on t.account_id = c.account_id and source_system_type = 'ticketing'
    join ticketing_events as te on te.id = t.event_id
)

select c.*, date_diff('day', first_purchase_date, CURRENT_DATE) as tenure
from (
    select 
        scv_id, 
        distance_to_venue, 
        stlr_product, 
        season_year, 
        max(event_date) as last_event_date,
        count(*) as tickets_purchased, 
        sum(price) as total_spent, 
        (1.0 * sum(case when was_attended then 1 else 0 end)) / count(*) as attendance_percentage,
        max(case when was_attended then event_date else null end) as last_attendance_date,
        (lead(count(*), 1) over (partition by scv_id order by season_year) is not null) as is_next_year_buyer
    from ticketing_customers
    group by scv_id, distance_to_venue, stlr_product, season_year) as c
join (
    select scv_id, min(purchased_at) as first_purchase_date
    from ticketing_customers
    group by scv_id
) as t
on c.scv_id = t.scv_id"""

In [ ]:
df_kernel = wr.athena.read_sql_query(
    boto3_session= SESSION, 
    ctas_approach= False,
    database= DATABASE,
    s3_output= S3_BUCKET,
    sql= sql3
)
df_kernel.shape

(201687, 11)

In [ ]:
df_redshift = redshift_tools.get_retention_dataset(
    cluster = "prod-app",
    database =  "stlrmilb",
    lkupclientid = "15",
    start_year = 2010,
    end_year = 2023
)

df_redshift.shape

Authorized as AROASQ4JELIXYLYV6P4UV:gdonst@stellaralgo.com


(159283, 24)

In [ ]:
df_kernel.head()

,scv_id,distance_to_venue,stlr_product,season_year,last_event_date,tickets_purchased,total_spent,attendance_percentage,last_attendance_date,is_next_year_buyer,tenure
0,000d3732aecbb17de5443dd71fea2623,23.179834,Individual,2015,2015-09-05 00:00:00,1,14.0,1.000000,2015-09-05 00:00:00,True,2769
1,000d3732aecbb17de5443dd71fea2623,23.179834,Individual,2016,2016-08-27 00:00:00,6,85.0,1.000000,2016-08-27 00:00:00,False,2769
2,0011f620b54d21fad6b06d3f6fd6b89c,NaN,Individual,2022,2022-04-08 06:00:00,2,46.0,1.000000,2022-04-08 06:00:00,False,363
3,0013b2eeb0b15ac34461fb67f4d2b099,NaN,Individual,2022,2022-05-13 06:00:00,6,120.0,0.833333,2022-05-13 06:00:00,False,332
4,001a201f03ccf63a2684e723b11a5d8a,21.633013,Individual,2013,2013-05-24 00:00:00,4,40.0,1.000000,2013-05-24 00:00:00,False,3602


In [ ]:
df_redshift.head()

,lkupclientid,clientcode,dimcustomermasterid,year,productgrouping,totalspent,recentdate,attendancepercent,renewedbeforedays,source_tenure,tenure,disttovenue,recency,missed_games_1,missed_games_2,missed_games_over_2,forward_records,opentosendratio,clicktosendratio,clicktoopenratio,gender,phonecall,inperson_contact,isnextyear_buyer
0,15,VEGAS51S,739691640,2019,Group,14.0,1970-01-01,0.00,14,1460,14,17.40,0,1,0,0,0,0.00,0.00,0.00,Unknown,0,0,0
1,15,VEGAS51S,739446034,2019,Half Season,1100.0,2019-08-30,0.69,183,1825,328,3.35,1,3,3,1,8,0.00,0.00,0.00,Unknown,0,0,0
2,15,VEGAS51S,739641777,2016,Individual,41.0,2016-05-26,1.00,0,2555,49,19.15,0,0,0,0,0,0.00,0.00,0.00,Unknown,0,0,0
3,15,VEGAS51S,739692822,2019,Group,19240.0,2019-08-18,0.60,25,2190,835,10.92,0,0,0,1,0,0.00,0.00,0.00,Unknown,0,0,0
4,15,VEGAS51S,739580670,2022,Individual,75.0,1970-01-01,0.00,34,1460,1127,20.72,0,1,0,0,1,0.00,0.00,0.00,Unknown,0,0,0


In [ ]:
df_kernel.season_year.value_counts().sort_index()

2012        1
2013     8282
2014     9380
2015    11647
2016    12047
2017    13819
2018    13958
2019    39300
2020     6082
2021    36230
2022    41762
2023     9179
Name: season_year, dtype: Int64

In [ ]:
df_redshift.year.value_counts().sort_index()

2013     7692
2014     8619
2015    11169
2016    11403
2017    12859
2018    13271
2019    29934
2021    29520
2022    34816
Name: year, dtype: int64

In [ ]:
df_kernel["stlr_product"].value_counts()

Individual     155773
Group           24012
Comp             8896
Mini Plan        5365
Full Season      4539
Half Season      2321
Flex Plan         781
Name: stlr_product, dtype: Int64

In [ ]:
df_redshift["productgrouping"].value_counts()

Individual     128826
Group           22294
Mini Plan        3088
Full Season      2947
Half Season      1412
Flex Plan         716
Name: productgrouping, dtype: int64

In [ ]:
df_kernel["total_spent"].sum()

333108200.0

In [ ]:
df_redshift["totalspent"].sum()

43810838.449999996